In [ ]:
import pandas as pd

import nltk
nltk.download('all')

In [ ]:
! pip install -U spacy

In [ ]:
df = pd.read_csv('../input/ted-talks/ted_main.csv')

In [ ]:
descriptions = df["description"]
descriptions[0]

In [ ]:
from nltk.tokenize import word_tokenize

description_tokenized = []
for description in descriptions:
    description_tokenized.append(word_tokenize(description, language='english'))


In [ ]:
from nltk.corpus import stopwords

english_stops = set(stopwords.words('english'))

In [ ]:
def remove_stop_words(description):
        return [word for word in description if word.lower() not in english_stops]

In [ ]:
text_token_stopwords = [remove_stop_words(description) for description in description_tokenized]

In [ ]:
text_token_stopwords